In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
imdb_reviews=pd.read_csv("/content/imdb_reviews.csv")
test_reviews=pd.read_csv("/content/test_reviews.csv")
imdb_reviews.head(10)

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative
5,<START begins better than it ends funny that t...,negative
6,<START lavish production values and solid perf...,positive
7,<START the <UNK> tells the story of the four h...,negative
8,<START just got out and cannot believe what a ...,positive
9,<START this movie has many problem associated ...,negative


In [5]:
test_reviews.head(10)

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive
5,<START i'm absolutely disgusted this movie isn...,positive
6,<START originally supposed to be just a part o...,positive
7,<START the <UNK> richard <UNK> dog is <UNK> to...,negative
8,<START hollywood had a long love affair with b...,negative
9,<START this film is where the batman franchise...,positive


In [20]:
word_index=pd.read_csv("/content/word_indexes.csv")
word_index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


In [7]:
word_index=dict(zip(word_index.Words,word_index.Indexes))
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3


In [8]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

In [9]:
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [10]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

In [11]:

train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)
train_data.head()

0    [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1    [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2    [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3    [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4    [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
Name: Reviews, dtype: object

In [12]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)


In [13]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [14]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

In [15]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 3s 43ms/step - loss: 0.6917 - accuracy: 0.5434 - val_loss: 0.6893 - val_accuracy: 0.6472
Epoch 2/30
49/49 [==============================] - 2s 39ms/step - loss: 0.6826 - accuracy: 0.6816 - val_loss: 0.6748 - val_accuracy: 0.7003
Epoch 3/30
49/49 [==============================] - 2s 39ms/step - loss: 0.6578 - accuracy: 0.7353 - val_loss: 0.6419 - val_accuracy: 0.7656
Epoch 4/30
49/49 [==============================] - 2s 39ms/step - loss: 0.6123 - accuracy: 0.7829 - val_loss: 0.5934 - val_accuracy: 0.7772
Epoch 5/30
49/49 [==============================] - 2s 38ms/step - loss: 0.5524 - accuracy: 0.8124 - val_loss: 0.5353 - val_accuracy: 0.8008
Epoch 6/30
49/49 [==============================] - 2s 39ms/step - loss: 0.4887 - accuracy: 0.8366 - val_loss: 0.4793 - val_accuracy: 0.8299
Epoch 7/30
49/49 [==============================] - 2s 38ms/step - loss: 0.4317 - accuracy: 0.8571 - val_loss: 0.4342 - val_accuracy: 0.8462
Epoch 8/30
49

In [17]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 2s 2ms/step - loss: 0.2966 - accuracy: 0.8849


In [18]:
index=np.random.randint(1,1000)
user_review=test_reviews.loc[index]
print(user_review)


Reviews      <START i saw this film at temple university i ...
Sentiment                                             negative
Name: 655, dtype: object


In [21]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")

1/1 [==============================] - 0s 19ms/step
negative sentiment


Now we can use this model, that is able to detection the sentiment of the text and feedback.
